## Get Sizing for 5G Speed Tiers Propensity for various segments

In [1]:
import pandas as pd
import numpy as np

### Ask from Chris -> 5G Dwell 

In [2]:
%%bigquery df

SELECT 
    A.*
FROM `roaming-pr-66a1b0.5g_upsell.bq_5g_3500_zone_subs` A



Query complete after 0.00s: 100% 1/1 [00:00<00:00, 914.99query/s] 
Downloading: 100% 1843345/1843345 [00:01<00:00, 1063905.52rows/s]


In [3]:
df

,imsi_id,msisdn_num,imei_id,is_5g_capable,total_dwell_duration_minute,unique_days,average_daily_dwell_duration_minute
0,302220316261052,2045832308,356765082957901,False,2130,7,304.285714
1,302220312164422,2049902611,356868112574439,False,1245,7,177.857143
2,302220319489316,2263440757,357207094132248,False,7305,7,1043.571429
3,302220546632937,2367777199,356171098077000,False,3450,7,492.857143
4,302220040811616,2368895219,352977092466641,False,1050,7,150.000000
...,...,...,...,...,...,...,...
1843340,302220536935078,9023187405,355285773439847,True,7785,14,556.071429
1843341,302220324216338,9028021648,350639798595790,True,16965,14,1211.785714
1843342,302220040318461,9028023839,352060429405143,True,11295,14,806.785714
1843343,302220316512621,9029541653,353610583822811,True,2010,14,143.571429


In [4]:
%%bigquery prop_5g_df 
SELECT * FROM `cto-wln-sa-data-pr-bb5283.customer_personas_reports.speed_tiers_campaign_output_5G` B  

Query complete after 0.00s: 100% 1/1 [00:00<00:00, 987.82query/s] 
Downloading: 100% 3145496/3145496 [00:01<00:00, 1619363.49rows/s]


In [5]:
prop_5g_df

,BAN,MSISDN,IMSI,propensity_score,is_5g_capable,is_4g_capable
0,20576396,2506682205,302220034316272,1.636312,No,Yes
1,12940867,4039880885,302220039900029,3.118580,No,Yes
2,17864118,2502634491,302220319944099,2.098059,No,Yes
3,19633784,4036277620,302220040820923,2.215363,No,Yes
4,39482431,4317927299,302220320570634,1.871307,No,Yes
...,...,...,...,...,...,...
3145491,1393682,5146681834,302220011700627,1.543580,Unknown,Unknown
3145492,19088513,4182932871,302220034121715,1.632194,Unknown,Unknown
3145493,37471838,4317456745,302220041754654,1.431494,Unknown,Unknown
3145494,39969192,6729784161,302220551066157,1.513700,Unknown,Unknown


In [6]:
merged_df = pd.merge(df, prop_5g_df[['MSISDN', 'propensity_score']], how='inner', left_on='msisdn_num', right_on = 'MSISDN')

In [7]:
merged_df

,imsi_id,msisdn_num,imei_id,is_5g_capable,total_dwell_duration_minute,unique_days,average_daily_dwell_duration_minute,MSISDN,propensity_score
0,302220040811616,2368895219,352977092466641,False,1050,7,150.000000,2368895219,3.025867
1,302220040822396,2503193336,354861092508081,False,1005,7,143.571429,2503193336,2.991247
2,302220013832860,2503510382,356395100309511,False,1200,7,171.428571,2503510382,2.179175
3,302220020024700,2505038593,356832114257435,False,6450,7,921.428571,2505038593,6.126632
4,302220545344057,2507131332,866080040532991,False,5235,7,747.857143,2507131332,3.399074
...,...,...,...,...,...,...,...,...,...
652824,302220029026686,8674456560,356688114624150,True,16845,14,1203.214286,8674456560,2.075399
652825,302220032885494,8736641966,352349697452428,True,4980,14,355.714286,8736641966,1.978180
652826,302220032019622,9022189537,354818250143697,True,2940,14,210.000000,9022189537,3.544201
652827,302220320010376,9023021167,356722114786733,True,17130,14,1223.571429,9023021167,2.672458


In [8]:
%%bigquery plan_5g_df 

SELECT 
    pi_prod_instnc_resrc_str AS MSISDN,
    pp_recur_chrg_amt,
    pi_prim_pp_sls_start_ts,
    pp_catlg_itm_nm, 
    CASE 
        WHEN pp_catlg_itm_nm LIKE '%5G+%' THEN 'Yes'
        WHEN pp_catlg_itm_nm LIKE '%5G %' THEN 'Yes'
        ELSE 'No'
    END AS plan_5G_5G_plus

FROM `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE 
  prod_instnc_ts = (SELECT MAX(prod_instnc_ts) FROM `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` )  -- Get most recent date in snapshot table

Query complete after 0.00s: 100% 6/6 [00:00<00:00, 3572.15query/s]                        
Downloading: 100% 21560165/21560165 [00:10<00:00, 2130540.31rows/s]


Exclude customers with 5G Plan

In [9]:
plan_5g_df.loc[plan_5g_df.plan_5G_5G_plus=='No'].reset_index(drop=True).head()

,MSISDN,pp_recur_chrg_amt,pi_prim_pp_sls_start_ts,pp_catlg_itm_nm,plan_5G_5G_plus
0,7054654925,70.0,2021-10-12 00:00:00,"Unlimited 10 - Unlimited nationwide, talk and ...",No
1,9027910184,55.0,2021-08-01 00:00:00,"Unlimited 20 - Unlimited nationwide, talk and ...",No
2,4039935286,65.0,2021-09-26 00:00:00,L&R 10GB Non-share Unlimited nationwide talk a...,No
3,2506157850,65.0,2021-11-20 00:00:00,L&R 10GB Non-share Unlimited nationwide talk a...,No
4,4038047563,70.0,2021-09-24 00:00:00,L&R 10GB Non-share Unlimited nationwide talk a...,No


In [10]:
merged_df_2 = merged_df.loc[merged_df.msisdn_num.isin(plan_5g_df.loc[plan_5g_df.plan_5G_5G_plus=='No'].reset_index(drop=True)['MSISDN'])]

531k customers identified, split between 5G capable = True and False -> Figure out how many are high and how many are low propensity

In [11]:
merged_df_2

,imsi_id,msisdn_num,imei_id,is_5g_capable,total_dwell_duration_minute,unique_days,average_daily_dwell_duration_minute,MSISDN,propensity_score
0,302220040811616,2368895219,352977092466641,False,1050,7,150.000000,2368895219,3.025867
1,302220040822396,2503193336,354861092508081,False,1005,7,143.571429,2503193336,2.991247
2,302220013832860,2503510382,356395100309511,False,1200,7,171.428571,2503510382,2.179175
3,302220020024700,2505038593,356832114257435,False,6450,7,921.428571,2505038593,6.126632
4,302220545344057,2507131332,866080040532991,False,5235,7,747.857143,2507131332,3.399074
...,...,...,...,...,...,...,...,...,...
652824,302220029026686,8674456560,356688114624150,True,16845,14,1203.214286,8674456560,2.075399
652825,302220032885494,8736641966,352349697452428,True,4980,14,355.714286,8736641966,1.978180
652826,302220032019622,9022189537,354818250143697,True,2940,14,210.000000,9022189537,3.544201
652827,302220320010376,9023021167,356722114786733,True,17130,14,1223.571429,9023021167,2.672458


In [12]:
merged_df_2.groupby(['is_5g_capable'])[['propensity_score']].describe()

propensity_score                                          \
                         count      mean       std       min       25%   
is_5g_capable                                                            
False                 298345.0  2.828120  1.119828  1.297077  1.994097   
True                  221931.0  3.578991  1.423707  1.349046  2.563538   

                                              
                    50%       75%        max  
is_5g_capable                                 
False          2.566322  3.372859  12.943669  
True           3.257836  4.244713  24.450027

5G Customers in 3.5GHz area not on 5G Plan

In [13]:
merged_df_2.loc[(merged_df_2.is_5g_capable ==True) & (merged_df_2.propensity_score >= merged_df_2.propensity_score.quantile(0.5))].shape

(142264, 9)

In [14]:
merged_df_2.loc[(merged_df_2.is_5g_capable ==True) & (merged_df_2.propensity_score >= merged_df_2.propensity_score.quantile(0.5))].reset_index(drop=True).to_csv('TELUS_5G_segment_1_q4.csv', index=False)

4G Customers in 3.5GHz area not on 5G Plan

In [15]:
merged_df_2.loc[(merged_df_2.is_5g_capable ==False) & (merged_df_2.propensity_score >= merged_df_2.propensity_score.quantile(0.5))].shape

(117874, 9)

## Bell Zones

In [16]:
%%bigquery bell_df

SELECT * FROM `roaming-pr-66a1b0.5g_upsell.bq_5g_3500_bell_zone_subs`

Query complete after 0.00s: 100% 1/1 [00:00<00:00, 1076.84query/s]
Downloading: 100% 110759/110759 [00:00<00:00, 161737.75rows/s]


In [17]:
bell_df

,ban,msisdn_num,bus_prod_instnc_id,imei_id,is_5g_capable,billg_addr_postal_cd,billg_addr_province_cd,billg_addr_city_nm
0,22709539.0,6474646126,20576689,867686022796195,False,L0H1G0,ON,GORMLEY
1,16674465.0,6474531222,19525367,353237108416174,False,L0H1G0,ON,GORMLEY
2,4069811.0,6478837526,7763981,356477104198213,False,L0H1G0,ON,GORMLEY
3,4454839.0,9057169522,8170266,None,False,L0H1G0,ON,GORMLEY
4,21267057.0,9054240033,13121927,352696106230922,False,L0H1G0,ON,GORMLEY
...,...,...,...,...,...,...,...,...
110754,35766743.0,4376848684,34172443,356686116722600,True,M9W7J5,ON,TORONTO
110755,7491076.0,4168858097,720703,358503112474904,True,M9W7J5,ON,ETOBICOKE
110756,13711970.0,6472689552,11840873,352626110857295,True,M9W7J5,ON,ETOBICOKE
110757,35119934.0,6472981650,24332682,351109224754683,True,M9W7J5,ON,ETOBICOKE


In [18]:
bell_df.groupby(['billg_addr_city_nm'])[['msisdn_num']].count().reset_index().sort_values(by='msisdn_num', ascending=False)

,billg_addr_city_nm,msisdn_num
162,TORONTO,53903
96,MISSISSAUGA,11954
139,SCARBOROUGH,9869
110,NORTH YORK,8790
74,MARKHAM,5418
...,...,...
68,LONDON,1
133,SCAARBOROUGH,1
132,SARNIA,1
131,SARBOROUGH,1


In [30]:
merged_bell_df = pd.merge(bell_df, prop_5g_df[['MSISDN', 'propensity_score', 'IMSI']], how='inner', left_on='msisdn_num', right_on = 'MSISDN')

In [31]:
merged_bell_df.groupby(['billg_addr_city_nm'])[['MSISDN']].count().reset_index().sort_values(by='MSISDN', ascending=False)

,billg_addr_city_nm,MSISDN
160,TORONTO,48073
96,MISSISSAUGA,10721
139,SCARBOROUGH,8682
110,NORTH YORK,7778
74,MARKHAM,4928
...,...,...
123,REXDALE,1
63,KILBRIDE,1
65,LEDUC,1
68,LONDON,1


In [32]:
merged_bell_df

,ban,msisdn_num,bus_prod_instnc_id,imei_id,is_5g_capable,billg_addr_postal_cd,billg_addr_province_cd,billg_addr_city_nm,MSISDN,propensity_score,IMSI
0,22709539.0,6474646126,20576689,867686022796195,False,L0H1G0,ON,GORMLEY,6474646126,1.847128,302220016601109
1,16674465.0,6474531222,19525367,353237108416174,False,L0H1G0,ON,GORMLEY,6474531222,3.987748,302220034766131
2,4069811.0,6478837526,7763981,356477104198213,False,L0H1G0,ON,GORMLEY,6478837526,2.186803,302220034516912
3,4454839.0,9057169522,8170266,None,False,L0H1G0,ON,GORMLEY,9057169522,2.262920,302220034730077
4,21267057.0,9054240033,13121927,352696106230922,False,L0H1G0,ON,GORMLEY,9054240033,3.371937,302220029360429
...,...,...,...,...,...,...,...,...,...,...,...
98819,35766743.0,4376848684,34172443,356686116722600,True,M9W7J5,ON,TORONTO,4376848684,6.524244,302220318591141
98820,7491076.0,4168858097,720703,358503112474904,True,M9W7J5,ON,ETOBICOKE,4168858097,3.525227,302220010941932
98821,13711970.0,6472689552,11840873,352626110857295,True,M9W7J5,ON,ETOBICOKE,6472689552,4.414773,302220019112249
98822,35119934.0,6472981650,24332682,351109224754683,True,M9W7J5,ON,ETOBICOKE,6472981650,2.802741,302220321063402


Exclude customers on 5G Plan

In [33]:
merged_bell_df_2

,ban,msisdn_num,bus_prod_instnc_id,imei_id,is_5g_capable,billg_addr_postal_cd,billg_addr_province_cd,billg_addr_city_nm,MSISDN,propensity_score
0,22709539.0,6474646126,20576689,867686022796195,False,L0H1G0,ON,GORMLEY,6474646126,1.847128
2,4069811.0,6478837526,7763981,356477104198213,False,L0H1G0,ON,GORMLEY,6478837526,2.186803
3,4454839.0,9057169522,8170266,None,False,L0H1G0,ON,GORMLEY,9057169522,2.262920
4,21267057.0,9054240033,13121927,352696106230922,False,L0H1G0,ON,GORMLEY,9054240033,3.371937
7,31583294.0,4163036047,28982485,356423103436824,False,L0H1G0,ON,GORMLEY,4163036047,3.436372
...,...,...,...,...,...,...,...,...,...,...
98813,38831353.0,4165640997,38542224,356721116992489,True,M9W7J4,ON,ETOBICOKE,4165640997,4.615884
98815,7057326.0,4168952659,817714,351082772556968,True,M9W7J4,ON,TORONTO,4168952659,3.344988
98817,9794878.0,6478805748,9546660,352349691249333,True,M9W7J4,ON,ETOBICOKE,6478805748,3.537582
98818,12891546.0,6478820281,7604185,358284148983040,True,M9W7J5,ON,TORONTO,6478820281,3.572987


In [34]:

merged_bell_df_2 = merged_bell_df.loc[merged_bell_df.msisdn_num.isin(plan_5g_df.loc[plan_5g_df.plan_5G_5G_plus=='No'].reset_index(drop=True)['MSISDN'])]

In [35]:
merged_bell_df_2

,ban,msisdn_num,bus_prod_instnc_id,imei_id,is_5g_capable,billg_addr_postal_cd,billg_addr_province_cd,billg_addr_city_nm,MSISDN,propensity_score,IMSI
0,22709539.0,6474646126,20576689,867686022796195,False,L0H1G0,ON,GORMLEY,6474646126,1.847128,302220016601109
2,4069811.0,6478837526,7763981,356477104198213,False,L0H1G0,ON,GORMLEY,6478837526,2.186803,302220034516912
3,4454839.0,9057169522,8170266,None,False,L0H1G0,ON,GORMLEY,9057169522,2.262920,302220034730077
4,21267057.0,9054240033,13121927,352696106230922,False,L0H1G0,ON,GORMLEY,9054240033,3.371937,302220029360429
7,31583294.0,4163036047,28982485,356423103436824,False,L0H1G0,ON,GORMLEY,4163036047,3.436372,302220543863978
...,...,...,...,...,...,...,...,...,...,...,...
98813,38831353.0,4165640997,38542224,356721116992489,True,M9W7J4,ON,ETOBICOKE,4165640997,4.615884,302220041507406
98815,7057326.0,4168952659,817714,351082772556968,True,M9W7J4,ON,TORONTO,4168952659,3.344988,302220021396634
98817,9794878.0,6478805748,9546660,352349691249333,True,M9W7J4,ON,ETOBICOKE,6478805748,3.537582,302220021419943
98818,12891546.0,6478820281,7604185,358284148983040,True,M9W7J5,ON,TORONTO,6478820281,3.572987,302220544300198


In [36]:
merged_bell_df_2.loc[(merged_bell_df_2.is_5g_capable ==True) & (merged_bell_df_2.propensity_score >= merged_bell_df_2.propensity_score.quantile(0.5))].shape

(19933, 11)

In [37]:
merged_bell_df_2.loc[(merged_bell_df_2.is_5g_capable ==True) & (merged_bell_df_2.propensity_score >= merged_bell_df_2.propensity_score.quantile(0.5))].reset_index(drop=True).to_csv('BELL_5G_segment_2_q4.csv', index=False)

In [26]:
merged_bell_df_2.loc[(merged_bell_df_2.is_5g_capable ==False) & (merged_bell_df_2.propensity_score >= merged_bell_df_2.propensity_score.quantile(0.5))].shape

(20280, 10)